In [1]:
%%capture

!wget 'https://github.com/CLC-HCMUS/ViMs-Dataset/raw/master/ViMs.zip'
!unzip 'ViMs.zip'

# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

In [2]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *

In [6]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wsegr,pos,ner", max_heap_size='-Xmx2g')

KeyboardInterrupt: 

In [ ]:
pathfiles = list()
for pathdir in glob.glob('/content/ViMs/original/*'):
  for pathfile in glob.glob(pathdir + '/original/*'):
    pathfiles.append(pathfile)

In [ ]:


# Văn bản toa thuốc tiếng Việt
text = "Bệnh nhân Nguyễn Văn A được kê đơn thuốc Paracetamol vào ngày 15/10/2024."

# Phân tích văn bản bằng VnCoreNLP
output = rdrsegmenter.annotate(text)

# Hiển thị kết quả nhận diện thực thể
for sentence in output['sentences']:
    for word_info in sentence:
        word = word_info[0]
        ner_label = word_info[-1]
        if ner_label != 'O':  # Nếu là thực thể (PER: tên người, DATE: ngày tháng)
            print(f"Word: {word}, Entity: {ner_label}")

# Hàm khử định danh
def deidentify_text_vietnamese(text, annotations):
    deidentified_text = text
    for sentence in annotations['sentences']:
        for word_info in sentence:
            word = word_info[0]
            ner_label = word_info[-1]
            if ner_label == 'B-PER':  # Nhận diện tên người
                deidentified_text = deidentified_text.replace(word, "[TÊN]")
            elif ner_label == 'B-DATE':  # Nhận diện ngày tháng
                deidentified_text = deidentified_text.replace(word, "[NGÀY]")
    return deidentified_text

# Khử định danh văn bản
deidentified_text = deidentify_text_vietnamese(text, output)
print(deidentified_text)
